# Importing relevant pacanges

In [37]:
import pandas as pd
from sqlalchemy import create_engine

# Importing Data from Databases
## Introduction to databases

Download the data from [github source](https://github.com/nabilseid/Data-Galaxy/blob/main/datacamp-de-notes/02-streamlined-data-ingestion-with-pandas/data.db) or [dataCamp source](https://assets.datacamp.com/production/repositories/4412/datasets/86d5855fd30d02afe8cb563da6057190694c6b86/data.db) and store it in the same directory with this notebook.

In [38]:
engine = create_engine('sqlite:///data.db')

In [39]:
weather = pd.read_sql('weather', engine)
weather_2 = pd.read_sql('select * from weather', engine)

In [40]:
weather.head()

,station,name,latitude,longitude,elevation,date,month,awnd,prcp,snow,tavg,tmax,tmin
0,USW00094728,"NY CITY CENTRAL PARK, NY US",40.77898,-73.96925,42.7,12/01/2017,December,5.37,0.00,0.0,,52,42
1,USW00094728,"NY CITY CENTRAL PARK, NY US",40.77898,-73.96925,42.7,12/02/2017,December,3.13,0.00,0.0,,48,39
2,USW00094728,"NY CITY CENTRAL PARK, NY US",40.77898,-73.96925,42.7,12/03/2017,December,2.01,0.00,0.0,,48,42
3,USW00094728,"NY CITY CENTRAL PARK, NY US",40.77898,-73.96925,42.7,12/04/2017,December,3.58,0.00,0.0,,51,40
4,USW00094728,"NY CITY CENTRAL PARK, NY US",40.77898,-73.96925,42.7,12/05/2017,December,6.71,0.75,0.0,,61,50


## Refining imports with SQL queries

`WHERE` clause with `AND` query.

In [41]:
and_query = """SELECT *
                 FROM hpd311calls
               WHERE borough = 'BRONX'
                 AND complaint_type = 'PLUMBING'"""

bx_plumbing_calls = pd.read_sql(and_query, engine)

bx_plumbing_calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2016 entries, 0 to 2015
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   unique_key        2016 non-null   object
 1   created_date      2016 non-null   object
 2   agency            2016 non-null   object
 3   complaint_type    2016 non-null   object
 4   incident_zip      2016 non-null   object
 5   incident_address  2016 non-null   object
 6   community_board   2016 non-null   object
 7   borough           2016 non-null   object
dtypes: object(8)
memory usage: 126.1+ KB


`WHERE` clause with `OR` query.

In [42]:
or_query = """SELECT *
                 FROM hpd311calls
               WHERE complaint_type = 'WATER LEAK'
                 OR complaint_type = 'PLUMBING'"""

leaks_or_plumbing = pd.read_sql(or_query, engine)

leaks_or_plumbing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10684 entries, 0 to 10683
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   unique_key        10684 non-null  object
 1   created_date      10684 non-null  object
 2   agency            10684 non-null  object
 3   complaint_type    10684 non-null  object
 4   incident_zip      10684 non-null  object
 5   incident_address  10684 non-null  object
 6   community_board   10684 non-null  object
 7   borough           10684 non-null  object
dtypes: object(8)
memory usage: 667.9+ KB


## More complex SQL queries

### Getting DISTINCT values
Get unique values of one or more columns

In [43]:
distinct_query = """SELECT DISTINCT incident_address, borough FROM hpd311calls"""

pd.read_sql(distinct_query, engine)

,incident_address,borough
0,2786 JEROME AVENUE,BRONX
1,323 EAST 12 STREET,MANHATTAN
2,1235 GRAND CONCOURSE,BRONX
3,656 WEST 171 STREET,MANHATTAN
4,1030 PARK PLACE,BROOKLYN
...,...,...
20135,22-16 74 STREET,QUEENS
20136,47 PERRY STREET,MANHATTAN
20137,1105 ELDER AVENUE,BRONX
20138,1137 PRESIDENT STREET,BROOKLYN


`hpd311calls` table has 20140 unique incident_address and borough value pairs

### Aggregate function

SQL support SUM, AVG, MIN, MAX, COUNT aggregate functions.

In [44]:
avg_query = """SELECT AVG(tmax) FROM weather"""

pd.read_sql(avg_query, engine)

,AVG(tmax)
0,43.504132


Average value of all tmax records is 43.50

In [45]:
count_query = """SELECT COUNT(*) FROM weather"""

pd.read_sql(count_query, engine)

,COUNT(*)
0,121


`weather` table has 121 records / rows. `COUNT()` is used to count the number of rows found for a query condition.

### Group By
To summarize data by category add `GROUP BY` clause and apply aggregate function.

In [46]:
group_by_query = """SELECT borough, COUNT(*)
                    FROM hpd311calls
                    WHERE complaint_type = 'PLUMBING'
                    GROUP BY borough;"""

pd.read_sql(group_by_query, engine)

,borough,COUNT(*)
0,BRONX,2016
1,BROOKLYN,2702
2,MANHATTAN,1413
3,QUEENS,808
4,STATEN ISLAND,178


When aggregate functions team with `GROUP BY`, they give summary on each category. Like above when `COUNT` team with `GROUP BY` we get count of rows in each groups.

## Loading multiple tables with joins

### Joining tables

In [47]:
join_by_query = """SELECT *
                        FROM hpd311calls
                            JOIN weather
                            ON hpd311calls.created_date == weather.date;"""

pd.read_sql(join_by_query, engine)

,unique_key,created_date,agency,complaint_type,incident_zip,incident_address,community_board,borough,station,name,...,longitude,elevation,date,month,awnd,prcp,snow,tavg,tmax,tmin
0,38070822,01/01/2018,HPD,HEAT/HOT WATER,10468,2786 JEROME AVENUE,07 BRONX,BRONX,USW00094728,"NY CITY CENTRAL PARK, NY US",...,-73.96925,42.7,01/01/2018,January,7.83,0.0,0.0,,19,7
1,38065299,01/01/2018,HPD,PLUMBING,10003,323 EAST 12 STREET,03 MANHATTAN,MANHATTAN,USW00094728,"NY CITY CENTRAL PARK, NY US",...,-73.96925,42.7,01/01/2018,January,7.83,0.0,0.0,,19,7
2,38066653,01/01/2018,HPD,HEAT/HOT WATER,10452,1235 GRAND CONCOURSE,04 BRONX,BRONX,USW00094728,"NY CITY CENTRAL PARK, NY US",...,-73.96925,42.7,01/01/2018,January,7.83,0.0,0.0,,19,7
3,38070264,01/01/2018,HPD,HEAT/HOT WATER,10032,656 WEST 171 STREET,12 MANHATTAN,MANHATTAN,USW00094728,"NY CITY CENTRAL PARK, NY US",...,-73.96925,42.7,01/01/2018,January,7.83,0.0,0.0,,19,7
4,38072466,01/01/2018,HPD,HEAT/HOT WATER,11213,1030 PARK PLACE,08 BROOKLYN,BROOKLYN,USW00094728,"NY CITY CENTRAL PARK, NY US",...,-73.96925,42.7,01/01/2018,January,7.83,0.0,0.0,,19,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91994,38353080,01/31/2018,HPD,HEAT/HOT WATER,10457,1511 SHERIDAN AVENUE,04 BRONX,BRONX,USW00094728,"NY CITY CENTRAL PARK, NY US",...,-73.96925,42.7,01/31/2018,January,6.71,0.0,0.0,,33,18
91995,38354647,01/31/2018,HPD,HEAT/HOT WATER,10457,1860 GRAND CONCOURSE,05 BRONX,BRONX,USW00094728,"NY CITY CENTRAL PARK, NY US",...,-73.96925,42.7,01/31/2018,January,6.71,0.0,0.0,,33,18
91996,38352537,01/31/2018,HPD,HEAT/HOT WATER,10462,2090 EAST TREMONT AVENUE,09 BRONX,BRONX,USW00094728,"NY CITY CENTRAL PARK, NY US",...,-73.96925,42.7,01/31/2018,January,6.71,0.0,0.0,,33,18
91997,38349554,01/31/2018,HPD,HEAT/HOT WATER,11213,187 ROCHESTER AVENUE,08 BROOKLYN,BROOKLYN,USW00094728,"NY CITY CENTRAL PARK, NY US",...,-73.96925,42.7,01/31/2018,January,6.71,0.0,0.0,,33,18


This query joins the two tables on recoreds that match on created_date from `hpd311calls` to date from `weather`

### Joining and filtering

In [48]:
join_n_filtering_by_query = """SELECT *
                                   FROM hpd311calls
                                       JOIN weather
                                       ON hpd311calls.created_date == weather.date
                                   WHERE hpd311calls.complaint_type == 'HEAT/HOT WATER';"""

pd.read_sql(join_n_filtering_by_query, engine)

,unique_key,created_date,agency,complaint_type,incident_zip,incident_address,community_board,borough,station,name,...,longitude,elevation,date,month,awnd,prcp,snow,tavg,tmax,tmin
0,38070822,01/01/2018,HPD,HEAT/HOT WATER,10468,2786 JEROME AVENUE,07 BRONX,BRONX,USW00094728,"NY CITY CENTRAL PARK, NY US",...,-73.96925,42.7,01/01/2018,January,7.83,0.0,0.0,,19,7
1,38066653,01/01/2018,HPD,HEAT/HOT WATER,10452,1235 GRAND CONCOURSE,04 BRONX,BRONX,USW00094728,"NY CITY CENTRAL PARK, NY US",...,-73.96925,42.7,01/01/2018,January,7.83,0.0,0.0,,19,7
2,38070264,01/01/2018,HPD,HEAT/HOT WATER,10032,656 WEST 171 STREET,12 MANHATTAN,MANHATTAN,USW00094728,"NY CITY CENTRAL PARK, NY US",...,-73.96925,42.7,01/01/2018,January,7.83,0.0,0.0,,19,7
3,38072466,01/01/2018,HPD,HEAT/HOT WATER,11213,1030 PARK PLACE,08 BROOKLYN,BROOKLYN,USW00094728,"NY CITY CENTRAL PARK, NY US",...,-73.96925,42.7,01/01/2018,January,7.83,0.0,0.0,,19,7
4,38076151,01/01/2018,HPD,HEAT/HOT WATER,10463,3810 BAILEY AVENUE,08 BRONX,BRONX,USW00094728,"NY CITY CENTRAL PARK, NY US",...,-73.96925,42.7,01/01/2018,January,7.83,0.0,0.0,,19,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55232,38353080,01/31/2018,HPD,HEAT/HOT WATER,10457,1511 SHERIDAN AVENUE,04 BRONX,BRONX,USW00094728,"NY CITY CENTRAL PARK, NY US",...,-73.96925,42.7,01/31/2018,January,6.71,0.0,0.0,,33,18
55233,38354647,01/31/2018,HPD,HEAT/HOT WATER,10457,1860 GRAND CONCOURSE,05 BRONX,BRONX,USW00094728,"NY CITY CENTRAL PARK, NY US",...,-73.96925,42.7,01/31/2018,January,6.71,0.0,0.0,,33,18
55234,38352537,01/31/2018,HPD,HEAT/HOT WATER,10462,2090 EAST TREMONT AVENUE,09 BRONX,BRONX,USW00094728,"NY CITY CENTRAL PARK, NY US",...,-73.96925,42.7,01/31/2018,January,6.71,0.0,0.0,,33,18
55235,38349554,01/31/2018,HPD,HEAT/HOT WATER,11213,187 ROCHESTER AVENUE,08 BROOKLYN,BROOKLYN,USW00094728,"NY CITY CENTRAL PARK, NY US",...,-73.96925,42.7,01/31/2018,January,6.71,0.0,0.0,,33,18


After joining the two tables filter the result where **complaint_type** is "HEAT/HOT WATER"

### Joining and aggregating

In [49]:
join_n_agg_by_query = """SELECT hpd311calls.borough,
                                count(*),
                                boro_census.total_population,
                                boro_census.housing_units
                         FROM hpd311calls
                             JOIN boro_census
                             ON hpd311calls.borough == boro_census.borough
                         GROUP BY hpd311calls.borough;"""

pd.read_sql(join_n_agg_by_query, engine)

,borough,count(*),total_population,housing_units
0,BRONX,29874,1455846,524488
1,BROOKLYN,31722,2635121,1028383
2,MANHATTAN,20196,1653877,872645
3,QUEENS,11384,2339280,850422
4,STATEN ISLAND,1322,475948,179179


After join the tables it group it by brought and do count on each category. When aggregating if a non aggregated column has multiple values the first record is taken.

# Importing JSON Data and Working with APIs
## Introduction to JSON
Data source for this section is the New York city leading causes of death. The data is publically available [Here](https://data.cityofnewyork.us/Health/New-York-City-Leading-Causes-of-Death/jb7j-dtam/data).
First we will load the json file and clean it up for pandas to understand. Then we will dump sub section of the data we are going to use.

In [ ]:
import json
from urllib.request import urlopen 

url_path = "https://data.cityofnewyork.us/api/views/jb7j-dtam/rows.json?accessType=DOWNLOAD"

with urlopen(url_path) as res:
    # load json file from url
    data = json.loads(res.read().decode())
    # extract column names from metadata with in the data
    columns = [column['name'] for column in data['meta']['view']['columns']]
    data = data['data']
    # reconstract the data in a format that pandas will understand
    nyc_death_causes = {'columns': columns, 'data': data}
    # convert dict type to string type then read it as a json
    nyc_death_causes_df = pd.read_json(json.dumps(nyc_death_causes), orient='split')
    # extract only the values we are interested in
    nyc_death_causes_df = nyc_death_causes_df[['Age Adjusted Death Rate', 
                                               'Death Rate', 'Deaths',
                                               'Leading Cause', 'Race Ethnicity',
                                               'Sex', 'Year']]
    # save the data in different json format
    nyc_death_causes_df.to_json('nyc_death_causes_split.json', orient='split')
    nyc_death_causes_df.to_json('nyc_death_causes_records.json', orient='records')
    nyc_death_causes_df.to_json('nyc_death_causes_index.json', orient='index')
    nyc_death_causes_df.to_json('nyc_death_causes_columns.json', orient='columns')
    nyc_death_causes_df.to_json('nyc_death_causes_values.json', orient='values')

**Reading different json orientations**

In [55]:
nyc_death_causes_split_df = pd.read_json('nyc_death_causes_split.json', orient='split')
nyc_death_causes_records_df = pd.read_json('nyc_death_causes_records.json', orient='records')
nyc_death_causes_index_df = pd.read_json('nyc_death_causes_index.json', orient='index')
nyc_death_causes_columns_df = pd.read_json('nyc_death_causes_columns.json', orient='columns')
nyc_death_causes_values_df = pd.read_json('nyc_death_causes_values.json', orient='values')

In [62]:
nyc_death_causes_split_df.head()

,Age Adjusted Death Rate,Death Rate,Deaths,Leading Cause,Race Ethnicity,Sex,Year
0,.,.,.,"Nephritis, Nephrotic Syndrome and Nephrisis (N...",Other Race/ Ethnicity,F,2009
1,18.5,16.3,204,Influenza (Flu) and Pneumonia (J09-J18),Hispanic,F,2013
2,.,.,.,"Assault (Homicide: Y87.1, X85-Y09)",Other Race/ Ethnicity,M,2012
3,.,.,5,Essential Hypertension and Renal Diseases (I10...,Not Stated/Unknown,F,2007
4,15.6,29.5,418,Cerebrovascular Disease (Stroke: I60-I69),White Non-Hispanic,F,2014


In [60]:
print(nyc_death_causes_split_df.equals(nyc_death_causes_records_df))
print(nyc_death_causes_split_df.equals(nyc_death_causes_index_df))
print(nyc_death_causes_split_df.equals(nyc_death_causes_columns_df))
print(nyc_death_causes_split_df.equals(nyc_death_causes_values_df))

True
True
True
False


In [63]:
nyc_death_causes_values_df.head()

,0,1,2,3,4,5,6
0,.,.,.,"Nephritis, Nephrotic Syndrome and Nephrisis (N...",Other Race/ Ethnicity,F,2009
1,18.5,16.3,204,Influenza (Flu) and Pneumonia (J09-J18),Hispanic,F,2013
2,.,.,.,"Assault (Homicide: Y87.1, X85-Y09)",Other Race/ Ethnicity,M,2012
3,.,.,5,Essential Hypertension and Renal Diseases (I10...,Not Stated/Unknown,F,2007
4,15.6,29.5,418,Cerebrovascular Disease (Stroke: I60-I69),White Non-Hispanic,F,2014
